# State Preparation with qclib

Qclib is a state preparation library built on Qiskit. The main focus is on preparing quantum states.
There is no direct documentation available online, but there is a Github page with source code:
- [qclib - Github](https://github.com/qclib/qclib)

And another Github page with links to example code and paper references (available on Arxiv)
- [qclib-papers - Github](https://github.com/qclib/qclib-papers/tree/main)

A few direct links to the papers:
- [Low-rank quantum state preparation](https://arxiv.org/abs/2111.03132)
- [Configurable sublinear circuits for quantum state preparation](https://arxiv.org/abs/2108.10182)
- [A divide-and-conquer algorithm for quantum state preparation](https://arxiv.org/abs/2008.01511)

In [1]:
import qclib

To learn how the library works, we can first look at the benchmarking script quoted in the `README` of the qclib repository: `https://github.com/qclib/qclib-papers/blob/main/examples/state_preparation_benchmark.ipynb`

#### Notes:
Some changes were made to conform to the newer V2.x version of Qiskit.

- [`QuantumCircuit.isometry`](https://docs.quantum.ibm.com/api/qiskit/0.25/qiskit.circuit.QuantumCircuit#isometry) has been superseded by [`qiskit.circuit.library.StatePreparation`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.StatePreparation)
- This is also the same method that [`QuantumCircuit.initialize`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#qiskit.circuit.QuantumCircuit.initialize) uses behind the scenes, so these are currently identical.

In [3]:
import numpy as np
from IPython.display import Markdown, display
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qclib.state_preparation import (
    LowRankInitialize,
    DcspInitialize,
    BdspInitialize,
    SVDInitialize,
    UCGInitialize
)
from qiskit.circuit.library import StatePreparation

n_qubits = 15

state = np.random.rand(2 ** n_qubits) + np.random.rand(2 ** n_qubits) * 1j
state = state/np.linalg.norm(state)

# qclib low-rank
circuit_low = QuantumCircuit(n_qubits)
LowRankInitialize.initialize(circuit_low, state)
# qclib svd
circuit_svd = QuantumCircuit(n_qubits)
SVDInitialize.initialize(circuit_svd, state)
# qclib ucg
circuit_ucg = QuantumCircuit(n_qubits)
UCGInitialize.initialize(circuit_ucg, state)
# qclib bdsp
circuit_bdsp = BdspInitialize(state).definition
# qclib dcsp
circuit_dcsp = DcspInitialize(state).definition
# qiskit StatePreparation
circuit_prep = StatePreparation(state).definition

In [ ]:
# print results
circuits = [('low-rank',    'qclib',  circuit_low),
            ('svd',         'qclib',  circuit_svd),
            ('ucg',         'qclib',  circuit_ucg),
            ('StatePreparation',    'qiskit', circuit_prep),
            ('bdsp',        'qclib',  circuit_bdsp),
            ('dcsp',        'qclib',  circuit_dcsp)]

table = '| method | lib | qubits | cnots | depth |\n'
table += '| --- | --- |:---:|:---:|:---:|\n'
for label, lib, circuit in circuits:
    try:
        transpiled = transpile(circuit, basis_gates=['u', 'cx'], optimization_level=0)
    except:
        continue

    qubits = len(transpiled.qubits)
    depth = transpiled.depth()
    cx = transpiled.count_ops().get('cx', 0)

    table += f'| {label} | {lib} | {qubits} | {cx} | {depth} |\n'

display(Markdown(table))

| method | lib | qubits | cnots | depth |
| --- | --- |:---:|:---:|:---:|
| svd | qclib | 15 | 38813 | 71580 |
| ucg | qclib | 15 | 32752 | 65505 |
| StatePreparation | qiskit | 15 | 32752 | 65505 |
| multiplexor | qiskit | 15 | 32752 | 65506 |
| bdsp | qclib | 1151 | 72320 | 1603 |
| dcsp | qclib | 32767 | 262016 | 899 |
